In [12]:
import os
import shutil
import pandas as pd
pd.set_option('display.max_colwidth', 100)


In [2]:
import faiss
assert faiss.get_num_gpus() > 0

In [3]:
# load topics from topics.txt with format
# <topics>
# 	<topic>
# 		<id>1</id>
# 		<query> What are the most common chronic diseases? What effects do chronic diseases have for the society and the individual?
# 		</query>
# 	</topic>
# 	<topic>
# 		<id>8</id>
# 		<query> best apps daily activity exercise diabetes
# 		</query>
# 	</topic>
# 	<topic>
# 		<id>22</id>
# 		<query> my risk for developing type 2 diabetes
# 		</query>
# 	</topic>
# </topics>

file_path = 'CHS-2021/topics/topics.txt'

with open(file_path, 'r') as f:
    topics = f.read()

topics = topics.split('<topic>')
topics = topics[1:]

topic_dict = {}
for topic in topics:
    topic_id = topic.split('<id>')[1].split('</id>')[0]
    topic_query = topic.split('<query>')[1].split('</query>')[0]
    topic_dict[topic_id] = topic_query

In [4]:

import pyterrier as pt
if not pt.started():
    pt.init()


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [5]:
assistant_results = { 
     'open-assistant-1': "The most common chronic diseases include heart disease, diabetes, cancer and arthritis. Chronic diseases also include dementia and Alzheimer's.  Chronic diseases effect not only individuals but entire populations as well. Many of these diseases can be prevented through healthy lifestyles such as regular exercise and good nutrition. The cost to treat chronic diseases is high both in terms of dollars spent on healthcare and loss of productivity due to illness or disability caused by the disease itself.",
     'open-assistant-2': "Here are some of the top-rated apps for tracking physical activity and managing blood sugar levels: 1. MyFitnessPal - a calorie tracker app that can help you track your food intake, exercise, and weight loss goals. It also has a feature to log glucose readings.  2. Fitbit - a wearable fitness tracker device with an accompanying app that tracks steps taken, distance covered, active minutes, calories burned, sleep, heart rate, and more. The app also allows users to set reminders to take medication or monitor their glucose levels throughout the day.  3. Lark DPP - A personalized digital Diabetes Prevention Program (DPP) designed to help people reduce their risk of developing type 2 diabetes through healthy lifestyle changes such as increased physical activity.  4. Glucose Buddy - is a simple but powerful app that helps you keep track of your glucose levels, carbohydrate consumption, insulin injections, and other important information related to managing your diabetes.  5. Glooko - is another popular diabetes management app that syncs directly with various types of glucometers and continuous glucose monitors from major manufacturers like Dexcom and Abbott. Users can view trends over time and share data with caregivers if needed.  6. SugarWise - uses advanced machine learning algorithms to analyze patterns and provide actionable recommendations based on the user's inputted meals and blood sugar measurements.  7. CarbMaster - a comprehensive app that provides detailed nutritional information about thousands of different food items, including serving sizes, macronutrient ratios, fiber content, and glycemic indices.  8. Blood Pressure Tracker - a dedicated app specifically geared towards monitoring blood pressure levels, complete with charts, graphs, and notifications for irregular readings.  9. Daily Burn - offers customizable workout plans tailored to specific interests and fitness levels, along with a meal planner tool and progress tracking capabilities. This app is great for those looking to mix up their exercises and incorporate strength training into t",
    'open-assistant-3': "The most important factors in determining the risk of getting Type 2 Diabetes are genetics and obesity. Genetic factors account to around 70-80% of the total risk, while being overweight accounts for another 15%. The remaining part is contributed by other environmental factors such as exercise habits and eating habits. Obese people who have a healthy lifestyle will still get Type 2 Diabetes but it will take longer compared to those who don't lead a healthier life style with similar BMI values. While this is generally true, there may be exceptions where even very thin or normal weighted people develop Type 2 Diabetes due to underlying hormonal disorders that affect metabolism.",
}

#load csv and add column names ['topic_id', 'query', 'answer']
chat_gpt_results = pd.read_csv('answers/chatgpt-16052023.csv', header=None)
chat_gpt_results.columns = ['topic_id', 'query', 'answer']
chat_gpt_results['topic_id'] = chat_gpt_results['topic_id'].astype(str)
# remove special characters and newlines from answer
chat_gpt_results['answer'] = chat_gpt_results['answer'].str.replace(r'[\n\r\t]', ' ').str.replace(r'[^\w\s]', ' ')
# only keep rows with answer
chat_gpt_results = chat_gpt_results[chat_gpt_results['answer'].notna()]

def ds_generate(expand_with_chatgpt=True):
        with open('CHS-2021/documents/Webdoc/crawl/txt.tsv', 'r') as corpusfile:
            for l in corpusfile:
                docno, passage = l.split("\t")
                yield {'docno' : docno, 'text' : passage}
            if expand_with_chatgpt:
                for idx, row in chat_gpt_results.iterrows():
                    docno = 'chatgpt' + str(row['topic_id'])
                    yield {'docno' : docno, 'text' : row['answer']}
# add preprocessor
use_colbert = True
index_path = './chatgpt_colbert_index'
if os.path.exists(index_path):  
    shutil.rmtree(index_path)
if use_colbert:
    from pyterrier_colbert.indexing import ColBERTIndexer

    checkpoint="http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip"
    indexer = ColBERTIndexer(checkpoint, index_path, "colbertindex", chunksize=3)
    indexref = indexer.index(ds_generate())
else:
    iter_indexer = pt.IterDictIndexer(index_path)
    indexref = iter_indexer.index(ds_generate(expand_with_chatgpt=True), meta={'docno' : 36, 'text': 2000})

/tmp/ipykernel_10789/3920195498.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  chat_gpt_results['answer'] = chat_gpt_results['answer'].str.replace(r'[\n\r\t]', ' ').str.replace(r'[^\w\s]', ' ')
/home/jonas/miniconda3/envs/faiss/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[May 17, 11:31:26] [0] 		 #> Local args.bsize = 128
[May 17, 11:31:26] [0] 		 #> args.index_root = ./chatgpt_colbert_index
[May 17, 11:31:26] [0] 		 #> self.possible_subset_sizes = [69905]


Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAI

[May 17, 11:31:30] #> Loading model checkpoint.
[May 17, 11:31:30] #> Loading checkpoint http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip


/home/jonas/miniconda3/envs/faiss/lib/python3.8/site-packages/torch/hub.py:665: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in the new zipfile format.
  warnings.warn('Falling back to the old format < 1.6. This support will be '


[May 17, 11:31:38] #> checkpoint['epoch'] = 0
[May 17, 11:31:38] #> checkpoint['batch'] = 44500




[May 17, 11:31:39] #> Creating directory ./chatgpt_colbert_index 




[May 17, 11:31:39] #> Creating directory ./chatgpt_colbert_index/colbertindex 


[May 17, 11:34:29] [0] 		 #> Completed batch #0 (starting at passage #0) 		Passages/min: 2.4k (overall),  2.4k (this encoding),  9922.7M (this saving)
[May 17, 11:34:29] [0] 		 [NOTE] Done with local share.
[May 17, 11:34:30] [0] 		 #> Joining saver thread.
[May 17, 11:34:30] [0] 		 #> Saved batch #0 to ./chatgpt_colbert_index/colbertindex/0.pt 		 Saving Throughput = 634.3k passages per minute.

#> num_embeddings = 1206320
[May 17, 11:34:30] #> Starting..
[May 17, 11:34:30] #> Processing slice #1 of 1 (range 0..1).
[May 17, 11:34:30] #> Will write to ./chatgpt_colbert_index/colbertindex/ivfpq.100.faiss.
[May 17, 11:34:30] #> Loading ./chatgpt_colbert_index/colbertindex/0.sample ...
#> Sample has shape (60316, 128)
[May 17, 11:34:30] Prepar

In [6]:
if use_colbert:
    from pyterrier_colbert.ranking import ColBERTFactory
    index=(index_path, "colbertindex")
    pytcolbert = ColBERTFactory(checkpoint, *index)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAI

[May 17, 11:34:56] #> Loading model checkpoint.
[May 17, 11:34:56] #> Loading checkpoint http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip
[May 17, 11:35:08] #> checkpoint['epoch'] = 0
[May 17, 11:35:08] #> checkpoint['batch'] = 44500


In [7]:
pytcolbert.index_path

'./chatgpt_colbert_index/colbertindex'

In [10]:
dense_e2e = pytcolbert.end_to_end()

[May 17, 12:20:16] #> Loading the FAISS index from ./chatgpt_colbert_index/colbertindex/ivfpq.faiss ..
[May 17, 12:20:16] #> Building the emb2pid mapping..
[May 17, 12:20:16] len(self.emb2pid) = 1206320
Loading reranking index, memtype=mem


Loading index shards to memory: 100%|██████████| 1/1 [00:00<00:00,  1.78shard/s]


In [14]:
dense_e2e.search("What are the most common chronic diseases? What effects do chronic diseases have for the society and the individual?").head(5)

,qid,query,docid,query_toks,query_embs,score,docno,rank
3028,1,What are the most common chronic diseases? What effects do chronic diseases have for the society...,6693,"[tensor(101), tensor(1), tensor(2054), tensor(2024), tensor(1996), tensor(2087), tensor(2691), t...","[[tensor(-0.0128), tensor(0.0504), tensor(-0.0809), tensor(0.1380), tensor(-0.0766), tensor(-0.0...",23.091539,chatgpt1,0
473,1,What are the most common chronic diseases? What effects do chronic diseases have for the society...,824,"[tensor(101), tensor(1), tensor(2054), tensor(2024), tensor(1996), tensor(2087), tensor(2691), t...","[[tensor(-0.0128), tensor(0.0504), tensor(-0.0809), tensor(0.1380), tensor(-0.0766), tensor(-0.0...",21.541136,eb767163-bf27-43dd-b3a8-50c21b3f4964,1
1772,1,What are the most common chronic diseases? What effects do chronic diseases have for the society...,3656,"[tensor(101), tensor(1), tensor(2054), tensor(2024), tensor(1996), tensor(2087), tensor(2691), t...","[[tensor(-0.0128), tensor(0.0504), tensor(-0.0809), tensor(0.1380), tensor(-0.0766), tensor(-0.0...",19.815054,c544fae2-f1ba-42a0-a9ff-db451c9545cc,2
2636,1,What are the most common chronic diseases? What effects do chronic diseases have for the society...,5125,"[tensor(101), tensor(1), tensor(2054), tensor(2024), tensor(1996), tensor(2087), tensor(2691), t...","[[tensor(-0.0128), tensor(0.0504), tensor(-0.0809), tensor(0.1380), tensor(-0.0766), tensor(-0.0...",19.380196,667fa530-c007-43af-83ee-e1aad46ee219,3
1427,1,What are the most common chronic diseases? What effects do chronic diseases have for the society...,3237,"[tensor(101), tensor(1), tensor(2054), tensor(2024), tensor(1996), tensor(2087), tensor(2691), t...","[[tensor(-0.0128), tensor(0.0504), tensor(-0.0809), tensor(0.1380), tensor(-0.0766), tensor(-0.0...",19.352016,8b822636-37ef-4d99-a02f-a004ee4835bd,4


In [15]:

import xml.etree.ElementTree as ET
import pandas as pd

def load_topics(path):
    with open(path) as f:
        topics = f.read()
    root = ET.fromstring(topics)
    dict = [(topic.find("id").text, topic.find("query").text.strip().lower()) for topic in root]
    return pd.DataFrame(dict, columns=["qid", "query"])

In [16]:

topics = load_topics("CHS-2021/topics/topics.txt")
qrels = pt.io.read_qrels("CHS-2021/assessments/qrels.txt")

# remove non alphanumeric characters from queries
topics["query"] = topics["query"].str.replace(r'\W+', ' ', regex=True)

In [ ]:
indexref.search("diabetes")

AttributeError: 'NoneType' object has no attribute 'search'

In [17]:
if not use_colbert:
    from pyterrier.measures import *
    bm25 = pt.BatchRetrieve(indexref, wmodel="BM25", metadata=["docno", "text"])  
    simpledph = pt.BatchRetrieve(indexref, wmodel="DPH", metadata=["docno", "text"])
    tfidf = pt.BatchRetrieve(indexref, wmodel="TF_IDF", metadata=["docno", "text"])

    bo1 = pt.rewrite.Bo1QueryExpansion(indexref)
    dph = pt.BatchRetrieve(indexref, wmodel="DPH", metadata=["docno", "text"])
    pipelineQE_dph = dph >> bo1 >> dph

    pipelineQE_bm25 = bm25 >> bo1 >> bm25

    pipelineQE_tfidf = tfidf >> bo1 >> tfidf

exp = pt.Experiment([dense_e2e],topics, qrels, ["map",  "bpref", "ndcg_cut_10"])

exp

,name,map,bpref,ndcg_cut_10
0,"Compose(pt.apply.generic(), pt.apply.by_query())",0.166916,0.377376,0.430149


In [22]:
# iterate over topics and save the positions of chatgpt answers
out_df = []
for idx, row in topics.iterrows():
    if str(row['qid']) not in chat_gpt_results['topic_id'].values:
        continue
    res = dense_e2e.search(row["query"])
    best_answer_text = res['docno'].tolist()[0]
    docno = 'chatgpt' + str(row['qid'])
    if docno not in res['docno'].values:
        out_df.append({'qid': row['qid'], 'query': row['query'], 'docno': docno, 'position': -1, 'chatgpt_answer': chat_gpt_results[chat_gpt_results['topic_id'] == str(row['qid'])]['answer'].values[0][:2000], 'best_answer': best_answer_text})
        continue
    position = res['docno'].tolist().index(docno)
    out_df.append({'qid': row['qid'], 'query': row['query'], 'docno': docno, 'position': position, 'chatgpt_answer': chat_gpt_results[chat_gpt_results['topic_id'] == str(row['qid'])]['answer'].values[0][:2000], 'best_answer': best_answer_text})
    
out_df = pd.DataFrame(out_df)

In [23]:
out_df

,qid,query,docno,position,chatgpt_answer,best_answer
0,1,what are the most common chronic diseases what effects do chronic diseases have for the society ...,chatgpt1,0,The most common chronic diseases vary depending on the country and population but some prevalen...,chatgpt1
1,8,best apps daily activity exercise diabetes,chatgpt8,4,There are several apps available that can help individuals with diabetes track their daily activ...,c67b03fe-95e8-40c4-9a68-339db80bb90c
2,22,my risk for developing type 2 diabetes,chatgpt22,8,As an AI language model I don t have access to personal medical information or the ability to d...,1a9ad7cb-345f-4964-adc3-cfb3994c5da6
3,35,is a ketogenic keto diet suitable for people with diabetes,chatgpt35,1,The ketogenic or keto diet is a high fat low carbohydrate diet that has been used primarily f...,ede2e34b-4d8c-4243-813d-83be732d99fc
4,45,can diabetes be cured,chatgpt45,2,As of my knowledge cutoff in September 2021 there is no known cure for diabetes Diabetes is a ...,38226475-9c84-45a7-ba0a-47d2f9be2493
5,51,what is holistic medicine and what does it include,chatgpt51,0,Holistic medicine also known as integrative or complementary medicine is an approach to health...,chatgpt51
6,52,what are the most common mental health problems how can i prevent or manage the most common ment...,chatgpt52,0,There are several common mental health problems that many individuals experience While it s imp...,chatgpt52
7,53,what is psychotherapy,chatgpt53,0,Psychotherapy also known as talk therapy or psychological therapy is a form of treatment that ...,chatgpt53
8,54,what does multiple sclerosis diagnosis include what does it change,chatgpt54,0,A multiple sclerosis MS diagnosis typically involves a combination of medical history evaluati...,chatgpt54
9,55,how to manage multiple sclerosis,chatgpt55,1,I m not a doctor but I can provide some general information on managing multiple sclerosis MS ...,053e0317-a7e4-412a-8a19-fb2e2110c97d


In [ ]:
# get chatgpt answer for each topic
out_df['answer'] = out_df['docno'].map(chat_gpt_results.set_index('topic_id')['answer'])
out_df

,qid,query,docno,position,chatgpt_answer,best_answer,answer
0,1,what are the most common chronic diseases what effects do chronic diseases have for the society and the individual,chatgpt1,3,The most common chronic diseases vary depending on the country and population but some prevalent examples worldwide include 1 Cardiovascular diseases These include conditions like hypertension high blood pressure coronary artery disease and stroke 2 Diabetes Both type 1 and type 2 diabetes are chronic diseases characterized by high blood sugar levels 3 Chronic respiratory diseases Conditions such as chronic obstructive pulmonary disease COPD asthma and pulmonary hypertension fall under this category 4 Cancer Different types of cancer such as lung breast colorectal and prostate cancer are chronic diseases that require long term management 5 Mental health disorders Mental illnesses like depression anxiety disorders bipolar disorder and schizophrenia are chronic conditions that impact mental well being 6 Chronic kidney disease This refers to long term damage to the kidneys leading to reduced function and potential kidney failure 7 Arthritis Conditions like osteoarthritis and rheumatoid arthritis cause joint inflammation and chronic pain 8 Neurological disorders Chronic diseases such as Alzheimer s disease Parkinson s disease and multiple sclerosis affect the nervous system and often have long lasting effects The effects of chronic diseases are significant both for individuals and society as a whole 1 Individual impact Chronic diseases can have a profound effect on individuals quality of life They may experience pain reduced mobility fatigue and limitations in daily activities Chronic conditions often require ongoing treatment management and lifestyle adjustments which can affect psychological well being and impose financial burdens due to healthcare costs 2 Societal impact Chronic diseases pose numerous challenges for society They contribute to a significant burden on healthcare systems consuming a substantial portion of healthcare budgets These diseases often require long term care including medicatio,"DermNet provides Google Translate, a free machine translation service. Note that this may not provide an exact translation in all languages This page was printed on 16/1/2021 DermNet NZ RANDOM RANDOM TRANSLATE TRANSLATE CLOSE SEARCH SEARCH DERMNET Home Topics AZ Images Browse DermDiag Glossary CME Quizzes About Contact Jobs Donate Home Topics AZ Images Browse DermDiag Glossary CME Quizzes About Contact Jobs Donate Home Topics AZ Chronic granulomatous disease Chronic granulomatous disease Author: Brian Wu PhD. MD Candidate, Keck School of Medicine, Los Angeles, USA; Chief Editor: Dr Amanda Oakley, Dermatologist, Hamilton, New Zealand, September 2015. Chronic granulomatous disease codes and concepts open Synonyms: Congenital dysphagocytosis, Chronic septic granulomatosis, Bridges Good syndrome, Quie syndrome Categories: Genetic disorder Subcategories: Primary immune deficiency, Clinical signs of chronic granulomatous disease ICD-10: D84.9, D84.8, D89.9 ICD-11: 4A00.0Y SNOMED CT: 387759001, 58606001, 234585008 What is chronic granulomatous disease? Chronic granulomatous disease is a primary immune deficiency due to functional defects in neutrophils and macrophages. It is characterised by: The inability of phagocytes cells that ingest microorganisms to make the chemicals necessary to kill off some types of fungi and bacteria The presence of granulomas, or aggregations of immune cells which cluster around sites that are inflamed or infected Inflammation, even in absence of infection The patients immune system is otherwise normal and the phagocytes are still capable of eliminating most pathogens. Chronic granulomatous disease is also called chronic septic granulomatosis, BridgesGood syndrome and Quie syndrome. Who gets chronic granulomatous disease? Chronic granulomatous disease is rare and is estimated to occur in approximately 1 in 250,000 to 500,0